In [1]:
import tensorflow as tf

## Introduction

In [2]:
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
x = tf.compat.v1.Variable(3, name="x")
y = tf.compat.v1.Variable(4, name="y")

In [4]:
f = x*x*y + y + 2

In [5]:
sess = tf.compat.v1.Session()

In [6]:
sess.run(x.initializer)

In [7]:
sess.run(y.initializer)

In [8]:
result = sess.run(f)

In [9]:
print(result)

42


In [10]:
sess.close()

In [11]:
with tf.compat.v1.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [12]:
init = tf.compat.v1.global_variables_initializer()

In [13]:
with tf.compat.v1.Session() as sess:
    init.run()
    result = f.eval()

In [14]:
sess = tf.compat.v1.InteractiveSession()

In [15]:
init.run()

In [16]:
result = f.eval()

In [17]:
print(result)

42


In [18]:
sess.close()

## Managing Graphs

In [19]:
x1 = tf.compat.v1.Variable(1)

In [20]:
x1.graph is tf.compat.v1.get_default_graph()

True

In [21]:
graph = tf.compat.v1.Graph()

In [22]:
with graph.as_default():
    x2 = tf.compat.v1.Variable(2)

In [23]:
x2.graph is tf.compat.v1.get_default_graph()

False

## Lifecycle of a node value

In [25]:
w = tf.compat.v1.constant(3)

In [26]:
x = w + 2
y = x + 5
z = x * 3

In [31]:
with tf.compat.v1.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [32]:
with tf.compat.v1.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression

In [33]:
import numpy as np

In [34]:
from sklearn.datasets import fetch_california_housing

In [35]:
housing = fetch_california_housing()

In [36]:
m,n = housing.data.shape

ERROR! Session/line number was not unique in database. History logging moved to new session 13


In [38]:
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [39]:
X = tf.compat.v1.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.compat.v1.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="Y")

In [40]:
XT = tf.transpose(X)

In [41]:
theta = tf.matmul(tf.matmul(tf.compat.v1.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [42]:
with tf.compat.v1.Session() as sess:
    theta_value = theta.eval()

## Manually computing Gradients

In [44]:
n_epochs = 1000
learning_rate = 0.1

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
sc = StandardScaler()

In [47]:
scaled_housing_data_plus_bias = sc.fit_transform(housing_data_plus_bias)

In [48]:
X = tf.compat.v1.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.compat.v1.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="Y")

In [49]:
theta = tf.compat.v1.Variable(tf.compat.v1.random_uniform([n+1,1],-1.0,1.0),name="theta")

In [50]:
y_pred = tf.matmul(X,theta, name="predictions")

In [51]:
error = y_pred - y

In [52]:
mse = tf.reduce_mean(tf.square(error), name="mse")

In [53]:
gradients = 2/m * tf.matmul(tf.transpose(X),error)

In [54]:
training_op = tf.compat.v1.assign(theta, theta - learning_rate*gradients)

In [55]:
init = tf.compat.v1.global_variables_initializer()

In [57]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch ",epoch,"MSE = ",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

ERROR! Session/line number was not unique in database. History logging moved to new session 17
Epoch  0 MSE =  6.3558774
Epoch  100 MSE =  4.8091307
Epoch  200 MSE =  4.803606
Epoch  300 MSE =  4.803289
Epoch  400 MSE =  4.803259
Epoch  500 MSE =  4.8032546
Epoch  600 MSE =  4.8032537
Epoch  700 MSE =  4.8032537
Epoch  800 MSE =  4.8032537
Epoch  900 MSE =  4.8032546


## Using Autodiff

In [59]:
def my_func(a,b):
    z = 0
    for i in range(100):
        z = a*np.cos(z+i)+z*np.sin(b-1)
    return z

In [60]:
gradients = tf.gradients(mse,[theta])[0]

## Using Optimizer

In [63]:
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)

In [64]:
training_op = optimizer.minimize(mse)

In [65]:
optimizer = tf.compat.v1.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

## Feeding data to training algorithm

In [66]:
A = tf.compat.v1.placeholder(tf.float32, shape=(None,3))
B = A + 5

In [67]:
with tf.compat.v1.Session() as sess:
    B_val_1 = B.eval(feed_dict={A : [[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A : [[4,5,6],[7,8,9]]})

In [68]:
print(B_val_1)

[[6. 7. 8.]]


In [69]:
print(B_val_2)

[[ 9. 10. 11.]
 [12. 13. 14.]]


In [70]:
X = tf.compat.v1.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="y")

In [71]:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [74]:
def fetch_batch(epoch,batch_index,batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [75]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()

In [78]:
def reset_graph(seed=42):
    tf.compat.v1.reset_default_graph()
    tf.compat.v1.set_random_seed(seed)
    np.random.seed(seed)

ERROR! Session/line number was not unique in database. History logging moved to new session 19


In [79]:
reset_graph()

In [81]:
n_epochs = 1000
learning_rate = 0.01

X = tf.compat.v1.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.compat.v1.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="Y")
theta = tf.compat.v1.Variable(tf.compat.v1.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred = tf.matmul(X,theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.compat.v1.global_variables_initializer()
saver = tf.compat.v1.train.Saver()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if(epoch%100==0):
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess,'/tmp/my_model.ckpt')
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess,"/tmp/my_model_final.ckpt")

Epoch 0 MSE = 7.0423403
Epoch 100 MSE = 4.9470916
Epoch 200 MSE = 4.8571677
Epoch 300 MSE = 4.8389177
Epoch 400 MSE = 4.8287582
Epoch 500 MSE = 4.8216386
Epoch 600 MSE = 4.8165174
Epoch 700 MSE = 4.812824
Epoch 800 MSE = 4.810159
Epoch 900 MSE = 4.8082366


In [82]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess,"/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [84]:
saver = tf.compat.v1.train.Saver({"weights":theta})

In [85]:
saver = tf.compat.v1.train.import_meta_graph("/tmp/my_model_final.ckpt.meta")

In [88]:
theta = tf.compat.v1.get_default_graph().get_tensor_by_name("theta:0") 

In [89]:
with tf.compat.v1.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")  # this restores the graph's state
    best_theta_restored = theta.eval() 

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


## Visualizing the graph

In [90]:
from datetime import datetime

In [91]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

In [93]:
root_logdir = "tf_logs"

In [94]:
logdir = "{}/run-{}".format(root_logdir,now)

ERROR! Session/line number was not unique in database. History logging moved to new session 23


In [99]:
n_epochs = 1000
batch_size = 100
n_batches = int(np.ceil(m / batch_size))


X = tf.compat.v1.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.compat.v1.Variable(tf.compat.v1.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred = tf.matmul(X,theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.compat.v1.global_variables_initializer()

ERROR! Session/line number was not unique in database. History logging moved to new session 26


In [100]:
mse_summary = tf.compat.v1.summary.scalar('MSE',mse)
file_writer = tf.compat.v1.summary.FileWriter(logdir,tf.compat.v1.get_default_graph())

In [101]:
with tf.compat.v1.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch,batch_index,batch_size)
            if batch_index%10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()

In [102]:
file_writer.close()